In [9]:
import os
from random import random

import cv2
import numpy as np
import pandas as pd

import image_processor
import model_builder

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.model_selection import train_test_split

from datetime import datetime

In [10]:
image_size = (32, 32)
input_size = (*image_size, 1)

In [11]:
df_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
            'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
            'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
            'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [12]:
df_train = image_processor.process_image_data_with_edges("data/Train", image_size, targets=df_names)

In [13]:
X = df_train.drop(columns=['target'])
y = df_train['target']

In [14]:
X_data = X.values.reshape(-1, *image_size, 1).astype('float32')
y_data = y.factorize()[0]

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.005, stratify=y_data, random_state=42)

In [16]:
model = model_builder.build_conv_pool_model(input_size, len(df_names))

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=256,
                    callbacks=[early_stopping, tensorboard_callback],
                    validation_data=(X_val, y_val),
                    verbose=1)

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv3d_1. Consider increasing the input size. Received input shape [None, 1, 64, 64, 1] which would produce output shape with a zero or negative value in a dimension.

In [16]:
for epoch, (val_loss, val_acc) in enumerate(zip(history.history['val_loss'], history.history['val_accuracy']), start=1):
    print(f"Epoch {epoch}: val_loss={val_loss:.4f}, val_accuracy={val_acc:.4f}")

score = model.evaluate(X_val, y_val, verbose=0)
print(f"Final Validation Accuracy: {score[1] * 100:.2f}%")

Epoch 1: val_loss=3.6545, val_accuracy=0.1165
Epoch 2: val_loss=0.7131, val_accuracy=0.7820
Epoch 3: val_loss=0.3730, val_accuracy=0.8797
Epoch 4: val_loss=0.3267, val_accuracy=0.8872
Epoch 5: val_loss=0.2982, val_accuracy=0.8835
Epoch 6: val_loss=0.2759, val_accuracy=0.8985
Epoch 7: val_loss=0.2736, val_accuracy=0.9173
Epoch 8: val_loss=0.2545, val_accuracy=0.8722
Epoch 9: val_loss=0.2436, val_accuracy=0.9023
Epoch 10: val_loss=0.2473, val_accuracy=0.8947
Epoch 11: val_loss=0.2529, val_accuracy=0.9173
Epoch 12: val_loss=0.2436, val_accuracy=0.8985
Epoch 13: val_loss=0.2331, val_accuracy=0.9098
Epoch 14: val_loss=0.3035, val_accuracy=0.8985
Epoch 15: val_loss=0.2968, val_accuracy=0.8947
Epoch 16: val_loss=0.2668, val_accuracy=0.9023
Epoch 17: val_loss=0.2430, val_accuracy=0.9135
Epoch 18: val_loss=0.2550, val_accuracy=0.9023
Final Validation Accuracy: 90.98%


In [17]:
test_df = df_test = image_processor.process_image_data_with_edges("data/TestData", image_size)

In [22]:
X_test = test_df.iloc[:, 1:]
X_test_prepared = X_test.values.reshape(-1, *image_size, 1).astype('float32')

In [23]:
pred = model.predict(X_test_prepared)

222/222 [==============================] - 1s 6ms/step


In [24]:
pred

array([[7.1457731e-12, 2.3294170e-12, 9.3129140e-01, ..., 8.0120060e-08,
        5.6839532e-08, 4.0921541e-03],
       [4.7364028e-09, 9.0312682e-08, 2.1152009e-10, ..., 1.6203050e-04,
        2.1481452e-07, 1.3879976e-05],
       [1.3601860e-08, 2.5647477e-09, 8.6971272e-12, ..., 6.0828124e-06,
        1.3891464e-09, 7.9736673e-10],
       ...,
       [7.4143580e-10, 1.7351707e-14, 1.2992955e-10, ..., 4.6022164e-16,
        2.1246162e-06, 9.6830747e-12],
       [2.4520563e-01, 4.9503862e-10, 9.7485763e-11, ..., 1.9674833e-10,
        3.5034334e-10, 4.0370691e-13],
       [2.3061357e-09, 1.5271134e-03, 5.9848936e-08, ..., 8.9920104e-06,
        2.7365263e-03, 4.7031220e-04]], dtype=float32)

In [26]:
predicted_classes = pred.argmax(axis=1)
len(predicted_classes)

7100

In [30]:
#C predicted_labels = [df_names[i] for i in predicted_classes]

test_df['pred'] = predicted_classes + 1

In [20]:
# model.save("models/model_final")

INFO:tensorflow:Assets written to: models/model_final/assets


INFO:tensorflow:Assets written to: models/model_final/assets


In [31]:
filtered_df = test_df[test_df['pred'] == 'l']
test_df[['label', 'pred']]

,label,pred
0,Test4751.png,3
1,Test2320.png,21
2,Test4989.png,24
3,Test6146.png,29
4,Test1629.png,55
...,...,...
7095,Test2339.png,52
7096,Test4990.png,24
7097,Test5456.png,10
7098,Test4748.png,25


In [38]:
df_lead = test_df[['pred', 'label']].sort_values(by=['label'], ascending=True).reset_index(drop=True)

In [39]:
df_lead.columns = ['class', 'TestImage']

In [40]:
df_lead

,class,TestImage
0,19,Test0001.png
1,19,Test0002.png
2,45,Test0003.png
3,45,Test0004.png
4,45,Test0005.png
...,...,...
7095,31,Test7096.png
7096,12,Test7097.png
7097,23,Test7098.png
7098,33,Test7099.png


In [43]:
df_lead.to_csv("vegleges_adat.csv", index=False, sep=';')